# Overview

This is my starter notebook for how to start with a recurrent neural network in this competition. It has much room for improvement, but hopefully this gives you a baseline to work with. 

This notebook currently score .142 on the latest version. I also have a lgbm training pipeline in this notebook that scores a .133 but it is not used in this inference.

The training pipelines are attached, the models were trained offline, and the weights are attached for inference.

## Model
- 300 feature input
- Batch Normalization
- Dense feature extractor with dropout
- Reshape and Batch Normalization to setup for RNN
- LSTM layers
- Dense head

See diagram below

## Callbacks
- ReduceLROnPlateau
- ModelCheckpoint
- EarlyStopping

## Future Ideas
- add new input features
- add investment_id as an input with an embedding layer
- use attention
- try different callbacks, learning rates, loss functions, etc
- try 1-D CNN

In [ ]:
from IPython.display import Image, display
display(Image(filename="../input/ubiquant-models/saved_models/images/rnn_v2.png", width = 210, height = 65))

In [ ]:
import pandas as pd
import numpy as np
import gc
import os
import pickle
from lightgbm import LGBMRegressor
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Cross Validation

In [ ]:
def setup_cv(df, X, y, groups, splits=5):
    kf = GroupKFold(n_splits=splits)
    for f, (t_, v_) in enumerate(kf.split(X=X, y=y, groups=groups)):
            df.loc[v_, 'fold'] = f

    return df

# Models

## RNN

In [ ]:
def get_rnn_v2():
    f300_in = L.Input(shape=(300,), name='300 feature input')
    x = L.BatchNormalization(name='batch_norm1')(f300_in)
    x = L.Dense(256, activation='swish', name='dense1')(x)
    x = L.Dropout(0.1, name='dropout1')(x)
    x = L.Reshape((1, -1), name='reshape1')(x)
    x = L.BatchNormalization(name='batch_norm2')(x)
    x = L.LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='relu', name='lstm1')(x)
    x = L.LSTM(16, dropout=0.1, return_sequences=False, activation='relu', name='lstm2')(x)
    output_layer = L.Dense(1, name='output')(x)

    model = M.Model([f300_in], 
                    [output_layer])

    model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
                  loss='mse', metrics=['mse'])

    return model

class UbiquantRNNV2:
    def __init__(self, df: pd.DataFrame, feature_cols: list=None, target: str='target'):

        self.model = get_rnn_v2()

        self.df = df

        if feature_cols is not None:
            self.feature_cols = feature_cols
        else:
            self.feature_cols = [f"f_{i}" for i in range(300)]

        self.target_col = target

    def train_one_fold(self, f: int, max_epochs=10):
        X_train = self.df[self.df.fold!=f][self.feature_cols]
        X_valid = self.df[self.df.fold==f][self.feature_cols]

        y_train = self.df[self.df.fold!=f][self.target_col]
        y_valid = self.df[self.df.fold==f][self.target_col]

        self.model.fit(X_train, y_train,
                       validation_data=(X_valid, y_valid),
                       batch_size=128, epochs=max_epochs,
                       callbacks=[
                         ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min'),
                         ModelCheckpoint(f'RNN_v2_checkpoint_{f}.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min'),
                         EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])

        oof = self.model.predict(X_valid)
        oof_score = np.sqrt(mean_squared_error(y_valid, oof))
        print(f'oof rmse: {oof_score}')

    def predict(self, X: np.ndarray):
        preds = self.model.predict(X)
        return preds

    def save(self, path: str):
        pickle.dump(self.model, open(path, 'wb'))

## LGBM

In [ ]:
class UbiquantLGBM:
    """
    This class is the Training Pipeline for an LGBM Regressor
    """
    def __init__(self, df: pd.DataFrame, feature_cols: list=None, target: str='target'):
        """ Creates the pipeline """
        params = {
            'random_state': 42, 
            'verbosity': -1,
            'metrics': 'rmse',
        }  
        self.model = LGBMRegressor(**params)

        self.df = df

        if feature_cols is not None:
            self.feature_cols = feature_cols
        else:
            self.feature_cols = [f"f_{i}" for i in range(300)]

        self.target_col = target

    def train_one_fold(self, f: int):
        """ Trains one fold of the lgbm """
        X_train = self.df[self.df.fold!=f][self.feature_cols]
        X_valid = self.df[self.df.fold==f][self.feature_cols]

        y_train = self.df[self.df.fold!=f][self.target_col]
        y_valid = self.df[self.df.fold==f][self.target_col]

        self.model.fit(X_train, y_train, 
                       eval_set=[(X_valid, y_valid)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=30)

        oof = self.model.predict(X_valid)
        oof_score = np.sqrt(mean_squared_error(y_valid, oof))
        print(f'oof rmse: {oof_score}')

    def predict(self, X: np.ndarray):
        """ Makes a prediction with the model """
        preds = self.model.predict(X)
        return preds

    def save(self, path: str):
        """Saves the model """
        pickle.dump(self.model, open(path, 'wb'))

# Inference

In [ ]:
def load_model(file_path):
    """ Loads a model pipeline object """
    file = open(file_path,'rb')
    model = pickle.load(file)
    file.close()
    return model

In [ ]:
""" Define our model with the trained weights """
rnn0 = get_rnn_v2()
rnn0.load_weights("../input/ubiquant-models/saved_models/rnn_checkpoints/RNN_v2_checkpoint_0.hdf5")
rnn1 = get_rnn_v2()
rnn1.load_weights("../input/ubiquant-models/saved_models/rnn_checkpoints/RNN_v2_checkpoint_1.hdf5")
rnn2 = get_rnn_v2()
rnn2.load_weights("../input/ubiquant-models/saved_models/rnn_checkpoints/RNN_v2_checkpoint_2.hdf5")
rnn3 = get_rnn_v2()
rnn3.load_weights("../input/ubiquant-models/saved_models/rnn_checkpoints/RNN_v2_checkpoint_3.hdf5")
rnn4 = get_rnn_v2()
rnn4.load_weights("../input/ubiquant-models/saved_models/rnn_checkpoints/RNN_v2_checkpoint_4.hdf5")

In [ ]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
f_col = df.drop(['row_id','time_id','investment_id','target'],axis=1).columns
f_col

In [ ]:
def make_dataset(df):
    inv_df = df['investment_id']
    f_df = df[f_col]
    scaled_investment_id = scaler.transform(pd.DataFrame(inv_df))
    df['investment_id'] = scaled_investment_id
    #data_x = pd.concat([df['investment_id'], f_df], axis=1)
    return f_df

In [ ]:
scaler = StandardScaler()
scaler.fit(pd.DataFrame(df['investment_id']))

In [ ]:
df=df.astype('float16')
df_x = make_dataset(df)
df_x

In [ ]:
df_y = pd.DataFrame(df['target'])
df_y

In [ ]:
del df

In [ ]:
kfold_generator = KFold(n_splits =5, shuffle=True, random_state = 2022)
kfold_generator

# Write your model name down in 'pythonash_model.h5'.
callbacks = tf.keras.callbacks.ModelCheckpoint('LSTM_model.h5', save_best_only = True)
for train_index, val_index in kfold_generator.split(df_x, df_y):
    # Split training dataset.
    train_x, train_y = df_x.iloc[train_index], df_y.iloc[train_index]
    # Split validation dataset.
    val_x, val_y = df_x.iloc[val_index], df_y.iloc[val_index]
    # Make tensor dataset.
    tf_train = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(2022).batch(1024, drop_remainder=True).prefetch(1)
    tf_val = tf.data.Dataset.from_tensor_slices((val_x, val_y)).shuffle(2022).batch(1024, drop_remainder=True).prefetch(1)
    # Load model
    model = get_rnn_v2()
    # Model fitting
    ## I used 5 epochs for fast save.
    ## Change the epochs into more numbers.
    model.fit(tf_train,
                       validation_data=(tf_val),
                       batch_size=128, epochs=10,
                       callbacks=[
                         ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min'),
                         EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])
    # Delete tensor dataset and model for avoiding memory exploring.
    del tf_train
    del tf_val
    del model

In [ ]:
""" Make predictions for competition """
import ubiquant
feats = [f"f_{i}" for i in range(300)]

#env = ubiquant.make_env()   
#iter_test = env.iter_test()    

for (test_df, sample_prediction_df) in iter_test:
    test_300 = test_df[feats]
    test_invest_id = test_df[['investment_id']]
    
    pred0 = rnn0.predict(test_300)
    pred1 = rnn1.predict(test_300)
    pred2 = rnn2.predict(test_300)
    pred3 = rnn3.predict(test_300)
    pred4 = rnn4.predict(test_300)
    pred = np.mean([pred0, pred1, pred2, pred3, pred4], axis=0)
    
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df)

In [ ]:
pd.read_csv("submission.csv")